In [4]:
!pip install faiss-cpu --no-cache

     |████████████████████████████████| 7.9MB 3.8MB/s 


In [86]:
import json

In [5]:
import faiss

In [9]:
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import keras.layers as L
from keras.models import Model
from keras import regularizers
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [12]:
dataset=pd.read_csv('/content/sample_data/prod_tasklist.csv', encoding='cp1252')

sentences=dataset['TASK'].to_list()

vocab_size = 5908
embedding_dim = 16
max_length = 10
trunc_type='post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words = vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

In [15]:
sequences = tokenizer.texts_to_sequences(sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

In [87]:
import json
import numpy as np
import codecs, json 

b = padded.tolist() # nested lists with same data, indices
file_path = "/padded.json" ## your path variable
json.dump(b, codecs.open(file_path, 'w', encoding='utf-8'), separators=(',', ':'), sort_keys=True, indent=4) ### this saves the array in .json format

In [83]:
padded

array([[  0,   0,   0, ..., 131, 373,  88],
       [  0,   0,   0, ...,   0, 143,  55],
       [  0,   0,   0, ..., 272,  47, 266],
       ...,
       [  0,   0,   0, ...,  50,  67,   1],
       [  0,   0,   0, ...,  50,  67,   1],
       [  0,   0,   0, ...,  50,  67,   1]], dtype=int32)

In [25]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(padded[1])
print(decode_review(padded[1]))
print(sentences[1])

[  0   0   0   0   0   0   0   0 143  55]
? ? ? ? ? ? ? ? patch management
Patch Management 


In [16]:
padded.shape

(16971, 10)

In [40]:
import numpy as np
import faiss


class FaissKNeighbors:
    def __init__(self, k=5):
        self.index = None
        self.y = None
        self.k = k

    def fit(self, X):
        self.index = faiss.IndexFlatL2(X.shape[1])
        self.index.add(X.astype(np.float32))
        

    def predict(self, X):
        distances, indices = self.index.search(X.astype(np.float32), k=self.k)
        # predictions = np.array([np.argmax(np.bincount(x)) for x in votes])
        return distances,indices

In [58]:
knn=FaissKNeighbors(k=10)
knn.fit(X=padded)

In [59]:
padded[1].shape

(10,)

In [92]:
i=1
distances,indices=knn.predict(np.expand_dims(padded[i],axis=0))
index_list=indices.flatten().tolist()
res_list = [sentences[i] for i in index_list] 
print('query search :',sentences[i])
print('recommended scripts',res_list)

query search : Patch Management 
recommended scripts ['Patch Management', 'Patch Management ', 'Patch Management', 'Patch Management ', 'Patch Management ', 'OS Installation', 'Process management', 'Reporting Automation', 'Server Patch Management', 'Database Patch Management']


In [82]:
list(set(res_list))[:5]

['Incident Management Report',
 'VMware Health Check',
 'SLA Daily Report',
 'Physical Memory Utilization',
 'Vmware Health Check']

In [48]:
index_list=indices.flatten().tolist()

[2672, 1, 13949, 8925, 2622]

In [51]:
index_list

[2672, 1, 13949, 8925, 2622]

In [66]:
a=np.array(sentences)
np.unique(a[index_list])

array(['Database Patch Management', 'OS Installation', 'Patch Management',
       'Patch Management ', 'Process management', 'Reporting Automation',
       'Server Patch Management'], dtype='<U506')

In [89]:
from sklearn.neighbors import NearestNeighbors
nei_clf = NearestNeighbors(p=2,metric='cosine')
nei_clf.fit(padded)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [94]:
distances,indices = nei_clf.kneighbors(np.expand_dims(padded[i],axis=0),n_neighbors=10)
index_list=indices.flatten().tolist()
res_list = [sentences[i] for i in index_list] 
print('query search :',sentences[i])
print('recommended scripts',list(set(res_list))[:5])

query search : Patch Management 
recommended scripts ['Backup Report', 'survey link', 'Restore Mail', 'Joiner - Reconciliation', 'Patch Management ']
